In [1]:
import folium
import os
import yaml
import numpy as np
import pandas as pd
import geopandas as gpd
from folium import plugins
from utils import prepare_input
from analysis import analysis
import matplotlib.pyplot as plt

In [2]:
with open(r'config.yaml') as file:
    model_params = yaml.load(file, Loader=yaml.FullLoader)

n_scenario = 192
num_scenario = '/' + str(n_scenario) + '_Scenario/'
model_params["path_to_input"] = os.path.dirname(os.getcwd()) + '/Data/fixed_reduced_grid' + num_scenario
model_params["input1"], model_params["input2"] = prepare_input(model_params["path_to_input"])

In [3]:
df = model_params["input1"].iloc[:,9:]
dominance_vector = np.ones(192)
columns = df.columns

for i in range(192):
    if dominance_vector[i] > 0:
        for j in range(i+1,192):
            temp1 = pd.Series(df[columns[i]] - df[columns[j]]).min()
            temp2 = pd.Series(df[columns[j]] - df[columns[i]]).min()

            if temp1 >= 0:
                dominance_vector[j] = 0
            elif temp2 >= 0:
                dominance_vector[i] = 0
            else:
                continue

In [4]:
non_dom_maps = []
for i in range(192):
    if dominance_vector[i] > 0:
        non_dom_maps.append(columns[int(i)])

In [5]:
non_dom_maps

['max_flood_level_wsw_5_05',
 'max_flood_level_w_5_05',
 'max_flood_level_w_5_10',
 'max_flood_level_w_5_15',
 'max_flood_level_w_5_25',
 'max_flood_level_wnw_5_05',
 'max_flood_level_wnw_5_10',
 'max_flood_level_wnw_5_15',
 'max_flood_level_wnw_5_25',
 'max_flood_level_nw_5_05',
 'max_flood_level_nw_5_10',
 'max_flood_level_nw_5_15',
 'max_flood_level_nw_5_25']

In [6]:
n_scenario = len(non_dom_maps)

num_scenario = '/' + str(n_scenario) + '_Scenario/'
dir_name = 'rm_' + str(n_scenario) +'/'

import os
import yaml
import json
import shutil
import numpy as np
from gurobipy import GRB
from utils import prepare_input
from main_model import two_stage_model

In [7]:
with open(r'config.yaml') as file:
    model_params = yaml.load(file, Loader=yaml.FullLoader)

model_params["path_to_input"] = "/Users/ashutoshshukla/Desktop/Data/fixed_reduced_grid/192_Scenario/"
model_params["path_to_output"] = os.getcwd() + "/output/" + dir_name

if os.path.exists(model_params["path_to_output"]):
    shutil.rmtree(model_params["path_to_output"], ignore_errors=True)
os.mkdir(model_params["path_to_output"])

model_params["input1"], model_params["input2"] = prepare_input(model_params["path_to_input"])

print("MIP-Gap: ", model_params["mip_gap"])
print("Time Limit: ", model_params["time_limit"])
print("Robust Model: ", model_params["robust_flag"])
print("Flexible Generation: ", model_params["flexible_generation"])
print("Objective Type:\t", model_params["set_objective"], "\n")
print("Solver Method:\t", model_params["solver_method"], "\n")

MIP-Gap:  0.005
Time Limit:  18000
Robust Model:  False
Flexible Generation:  True
Objective Type:	 min 

Solver Method:	 2 



In [8]:
for i in model_params["input1"].columns:
    if "flood" in i:
        if i not in non_dom_maps:
            model_params["input1"].drop([i], axis=1, inplace=True)

In [9]:
model_params["robust_flag"] = True

In [10]:
budget_vector = [0,10,20,30,40,50,60, 70, 80]
base_model = two_stage_model(model_params)

Set parameter Username
Academic license - for non-commercial use only - expires 2022-11-26


In [11]:
for budget in budget_vector:
    base_model.budget_ref.rhs = budget*1e6
    base_model.model.setParam("LogFile", model_params["path_to_output"] + str(budget) + "M")
    base_model.model.setParam("MIPGap", model_params["mip_gap"])
    base_model.model.setParam("TimeLimit", model_params["time_limit"])
    base_model.model.setParam("Method", model_params["solver_method"])
    base_model.model.optimize()
    base_model.model.write(model_params["path_to_output"] + str(budget) + "M_" + "solution.sol")    
    print("\n\n*********************************************")
    print("*********************************************\n\n")
    
    # budget_solution[budget]["objective_value"] = base_model.model.ObjVal
    # for i in base_model.x.keys():
    #     budget_solution[budget]['substation_' + str(i)] = base_model.x[i].X

Set parameter LogFile to value "/Users/ashutoshshukla/Desktop/TwoStageModel/output/rm_13/0M"
Set parameter MIPGap to value 0.005
Set parameter TimeLimit to value 18000
Set parameter Method to value 2
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[rosetta2])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 172210 rows, 63450 columns and 488191 nonzeros
Model fingerprint: 0xec6ae677
Variable types: 45488 continuous, 17962 integer (17600 binary)
Coefficient statistics:
  Matrix range     [1e-03, 1e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 3e+00]
  RHS range        [1e-02, 2e+04]
Presolve removed 161646 rows and 49102 columns
Presolve time: 0.30s
Presolved: 10564 rows, 14348 columns, 52870 nonzeros
Variable types: 12584 continuous, 1764 integer (1764 binary)
Found heuristic solution: objective 379.5386330
Root barrier log...

Ordering time: 0.01s

Barrier statistics:
 AA' NZ     : 6.239e+04
 Factor NZ  : 2.5

In [12]:
with open(model_params["path_to_output"] + 'model_params.json', 'w') as fp:
    del model_params["input1"]
    del model_params["input2"]
    json.dump(model_params, fp)